In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
sub_1 = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
train_df.head()

In [ ]:
train_df.shape

In [ ]:
y = train_df['Survived']
X = train_df.drop(columns=['Cabin','Name','Ticket','PassengerId','Survived'])
X.head()

In [ ]:
X.shape

In [ ]:
test_pid = test_df['PassengerId']
test_df.drop(columns = ['Cabin','Name','Ticket','PassengerId'],inplace=True)
test_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(pd.isna(X),yticklabels=False,cmap='viridis')

In [ ]:
X['Age'] = (X['Age'].fillna(X['Age'].mean()))
print(X.head())
sns.heatmap(pd.isna(X),yticklabels=False,cmap='viridis')

In [ ]:
sex_enc = (pd.get_dummies(X['Sex']))
embarked_enc = (pd.get_dummies(X['Embarked']))
print(sex_enc,embarked_enc)

In [ ]:
X = (pd.concat([X,sex_enc,embarked_enc],axis=1))


In [ ]:
X.shape

In [ ]:
X = (X.drop(columns = ['Sex','Embarked']))
X = (X.rename(columns ={'C':'Embarked_C','Q':'Embarked_Q','S':'Embarked_S'}))
X.head()

In [ ]:
X.columns

In [ ]:
for i in X.columns:
    print(type(X[i]))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X))

X = pd.DataFrame(scaler.transform(X))
X.head()

In [ ]:
X.shape,y.shape

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer,accuracy_score,f1_score,precision_score,roc_auc_score
rbf_clf = SVC(kernel = 'rbf')

C_range = 10.0 ** np.arange(-4, 4)
gamma_range = 10.0 ** np.arange(-4, 4)
param_grid = dict(gamma=gamma_range.tolist(), C=C_range.tolist())

scoring = {'accuracy': make_scorer(accuracy_score),'f1':make_scorer(f1_score),
           'AUC':make_scorer(roc_auc_score)}


grd_srch = GridSearchCV(rbf_clf,param_grid,scoring=scoring,refit = 'accuracy')
grd_srch.fit(X,y)
grd_srch.best_estimator_

In [ ]:
print(grd_srch.best_params_,grd_srch.best_score_)


In [ ]:
sns.heatmap(test_df.isna(),cmap='rainbow',yticklabels=False)

In [ ]:
test_df['Age'] = (test_df['Age'].fillna(test_df['Age'].mean()))
test_df['Fare'] = (test_df['Fare'].fillna(test_df['Fare'].mean()))

In [ ]:
sex_enc = (pd.get_dummies(test_df['Sex']))
embarked_enc = (pd.get_dummies(test_df['Embarked']))
test_df = (pd.concat([test_df,sex_enc,embarked_enc],axis=1))
test_df = (test_df.drop(columns = ['Sex','Embarked']))
test_df = (test_df.rename(columns ={'C':'Embarked_C','Q':'Embarked_Q','S':'Embarked_S'}))
test_df = pd.DataFrame(scaler.fit_transform(test_df))
test_df.head()



In [ ]:
y_test = grd_srch.predict(test_df)
y_test

In [ ]:
predictions =pd.DataFrame( np.column_stack((test_pid,y_test)) , columns = ['PassengerId','Survived'])
print(predictions.head())
sub_1.head()

In [ ]:
predictions.to_csv('kaggle_titanic_prediction_using_svm_rbf.csv',index = False)